In [1]:
import sys
sys.path.append('/mnt/d/ariel2/code/core/')
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
import ariel_load
import tqdm

local


In [2]:
%%time
train_data = kgs.load_all_train_data()
test_data = kgs.load_all_test_data()
len(train_data), len(test_data)

CPU times: user 556 ms, sys: 110 ms, total: 665 ms
Wall time: 1.85 s


(1099, 1)

In [ ]:
import ariel_simple2
importlib.reload(ariel_simple2)
model = ariel_simple2.SimpleModel()
model.do_plots=True
model.run_in_parallel = False    
model.train(train_data)
#model.bias_a = [0.,0.]
#model.bias_b = [0.,0.]
data = train_data[:1]
inferred_data = model.infer(data)

[   0   14   27   41   55   68   82   95  109  123  136  150  164  177
  191  204  218  232  245  259  273  286  300  313  327  341  354  368
  382  395  409  422  436  450  463  477  491  504  518  531  545  559
  572  586  600  613  627  640  654  668  681  695  709  722  736  749
  763  777  790  804  818  831  845  858  872  886  899  913  927  940
  954  967  981  995 1008 1022 1036 1049 1063 1076 1090 1104 1117 1131
 1145 1158 1172 1185 1199 1213 1226 1240 1254 1267 1281 1294 1308 1322
 1335 1349]
[450 463 477 491 504 518 531 545 559 572 586 600 613 627 640 654 668 681
 695 709 722 736 749 763 777 790 804 818 831 845 858 872 886 899]
[2.50276389 2.57498611 2.65276389 2.73054167 2.80276389 2.88054167
 2.95276389 3.03054167 3.10831944 3.18054167 3.25831944 3.33609722
 3.40831944 3.48609722 3.55831944 3.63609722 3.713875   3.78609722
 3.863875   3.94165278 4.013875   4.09165278 4.163875   4.24165278
 4.31943056 4.39165278 4.46943056 4.54720833 4.61943056 4.69720833
 4.76943056 4.847

In [6]:
%debug

> /mnt/d/ariel2/code/core/ariel_simple2.py(145)_infer_single()
    143                 t0_vals = np.round(np.linspace(0,len(self._times[ii]),self.t_steps)).astype(int)
    144                 print(t0_vals)
--> 145                 t0_vals = t0_vals[self._times[t0_vals]>self.min_t0 & [self._times[t0_vals]>self.max_t0]]
    146                 print(t0_vals)
    147                 res_mat = cp.empty((len(t0_vals), len(self.rp_vals)))



ipdb>  self._times[t0_vals]


*** TypeError: only integer scalar arrays can be converted to a scalar index


ipdb>  t0_vals


array([   0,   14,   27,   41,   55,   68,   82,   95,  109,  123,  136,
        150,  164,  177,  191,  205,  218,  232,  245,  259,  273,  286,
        300,  314,  327,  341,  355,  368,  382,  395,  409,  423,  436,
        450,  464,  477,  491,  505,  518,  532,  545,  559,  573,  586,
        600,  614,  627,  641,  655,  668,  682,  695,  709,  723,  736,
        750,  764,  777,  791,  805,  818,  832,  845,  859,  873,  886,
        900,  914,  927,  941,  955,  968,  982,  995, 1009, 1023, 1036,
       1050, 1064, 1077, 1091, 1105, 1118, 1132, 1145, 1159, 1173, 1186,
       1200, 1214, 1227, 1241, 1255, 1268, 1282, 1295, 1309, 1323, 1336,
       1350])


...

ipdb>  self._times.shape


*** AttributeError: 'list' object has no attribute 'shape'


ipdb>  self._times


[array([2.76388889e-03, 8.31944444e-03, 1.38750000e-02, ...,
       7.48609722e+00, 7.49165278e+00, 7.49715278e+00]), array([2.70833333e-03, 9.37500000e-03, 1.60416667e-02, ...,
       7.48270833e+00, 7.48937500e+00, 7.49537500e+00])]


ipdb>  q


In [ ]:
for ii in range(2):
    print(inferred_data[ii].diagnostics['t_ingress'],inferred_data[ii].diagnostics['t_egress'])
    plt.figure()
    plt.plot(model._times[0], model.pred[0])

In [ ]:
%debug

In [ ]:
import ariel_simple
importlib.reload(ariel_simple)
df = 0.05
for jj in [0]:
    model = ariel_simple.SimpleModel()
    model.run_in_parallel = True
    match jj:
        case 0:
            name = 'Default'
        case 1:
            name = 'Use correction factor'
            model.use_correction_factor = True        
    model.do_plots = False
    model.train(train_data)
    #model.bias_a = [0.,0.]
    #model.bias_b = [0.,0.]
    data = train_data
    inferred_data = model.infer(data)
    print(name)
    solution = kgs.make_submission_dataframe(data, include_sigma=False)
    submission = kgs.make_submission_dataframe(inferred_data, False)
    print(1e6*robust_linear_fit_rms(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy(), drop_frac=df), 1e6*robust_linear_fit_rms(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy(), drop_frac=0))
    print(1e6*robust_linear_fit_rms(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1),drop_frac=df),
         1e6*robust_linear_fit_rms(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1),drop_frac=0))
    kgs.sanity_checks['simple_residual_diff_FGS'] = kgs.SanityCheckValue('simple_residual_diff_FGS', 12, [-1,1])
    kgs.sanity_checks['simple_residual_diff_FGS'].seen_all = [d.diagnostics['simple_residual_diff_FGS'] for d in inferred_data]
    kgs.sanity_checks['simple_residual_diff_AIRS'] = kgs.SanityCheckValue('simple_residual_diff_AIRS', 12, [-1,1])
    kgs.sanity_checks['simple_residual_diff_AIRS'].seen_all = [d.diagnostics['simple_residual_diff_AIRS'] for d in inferred_data]
    kgs.dill_save(kgs.temp_dir + '/test_simple.pickle', (data,inferred_data,kgs.sanity_checks))

In [ ]:
print(1e6*robust_linear_fit_rms(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy(), drop_frac=df), 1e6*robust_linear_fit_rms(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy(), drop_frac=0))
print(1e6*robust_linear_fit_rms(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1),drop_frac=df),
    1e6*robust_linear_fit_rms(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1),drop_frac=0))

In [ ]:
score, rms_err = kgs.score_metric(inferred_data, data)
#np.sqrt(np.diag(inferred_data[0].spectrum_cov))

In [ ]:
import copy
data_sim = copy.deepcopy(data)
sig = rms_err
for ii,d in enumerate(data_sim):
    d.spectrum += np.random.default_rng(seed=ii).normal(0,sig,size=d.spectrum.shape)
    d.spectrum_cov = np.diag(0*d.spectrum+sig**2)
    d.check_constraints()
print('Theoretical')
kgs.score_metric(data_sim, data);

In [ ]:
kgs.print_sanity_checks()

In [ ]:
plt.figure();
plt.scatter(kgs.sanity_checks['simple_residual_diff_FGS'].seen_all, solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy())
plt.figure();
plt.scatter(kgs.sanity_checks['simple_residual_diff_AIRS'].seen_all, np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1))

In [ ]:
plt.figure()
plt.plot(kgs.sanity_checks['simple_residual_diff_FGS'].seen_all)
plt.plot(kgs.sanity_checks['simple_residual_diff_AIRS'].seen_all)
plt.legend(('FGS', 'AIRS'))
plt.grid(True)

In [ ]:
solution = kgs.make_submission_dataframe(data, include_sigma=False)
submission = kgs.make_submission_dataframe(inferred_data, False)


In [ ]:
kgs.rms(solution.iloc[:,1:2].to_numpy()-submission.iloc[:,1:2].to_numpy()),kgs.rms(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy())

In [ ]:
_,ax=plt.subplots(1,2,figsize=(10,5))
plt.sca(ax[0])
plt.box(True);plt.grid(True);
plt.scatter(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy())
plt.title(1e6*kgs.rms(solution.iloc[:,1:2].to_numpy()-submission.iloc[:,1:2].to_numpy()))
plt.sca(ax[1])
plt.box(True);plt.grid(True);
plt.scatter(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1))
plt.title(1e6*kgs.rms(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy()))

In [ ]:
ind = np.argmax(np.abs(np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1)))
ind

In [ ]:
plt.scatter([len(d.transits) for d in train_data],np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1))

In [ ]:
plt.scatter([d.i for d in train_data],kgs.sanity_checks['simple_residual_diff_AIRS'].seen_all)

In [ ]:
import ariel_simple
importlib.reload(ariel_simple)
df = 0.05
model = ariel_simple.SimpleModel()
model.run_in_parallel = True  
model.do_plots = False
#model.bias_a = [0.,0.]
#model.bias_b = [0.,0.]
data = train_data[:50]
model.train(data)
inferred_data = model.infer(data)
solution = kgs.make_submission_dataframe(data, include_sigma=False)
submission = kgs.make_submission_dataframe(inferred_data, False)
print(1e6*robust_linear_fit_rms(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy(), drop_frac=df), 1e6*robust_linear_fit_rms(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy(), drop_frac=0))
print(1e6*robust_linear_fit_rms(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1),drop_frac=df),
     1e6*robust_linear_fit_rms(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1),drop_frac=0))
kgs.score_metric(inferred_data,data)

In [ ]:
import ariel_model
importlib.reload(ariel_model)
df = 0.05
model = ariel_simple.SimpleModel()
model.run_in_parallel = True  
model.do_plots = False
model.bias_a = [0.,0.]
model.bias_b = [0.,0.]
model = ariel_model.Fudger(model=model)
data = train_data[:5]
print('5')
model.train(data)
inferred_data = model.infer(data)
solution = kgs.make_submission_dataframe(data, include_sigma=False)
submission = kgs.make_submission_dataframe(inferred_data, False)
print(1e6*robust_linear_fit_rms(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy(), drop_frac=df), 1e6*robust_linear_fit_rms(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy(), drop_frac=0))
print(1e6*robust_linear_fit_rms(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1),drop_frac=df),
     1e6*robust_linear_fit_rms(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1),drop_frac=0))
kgs.score_metric(inferred_data,data)

In [ ]:
model.sigma_fudge

In [ ]:
%debug